In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
class CustomModel(keras.Model):
    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True) 
            # 計算Loss
            # compile()的工作，根據計算Loss標準
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # 計算梯度
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # 更新權重
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # 更新與紀錄各類衡量標準，包含準確率、損失
        self.compiled_metrics.update_state(y, y_pred)
        # 回傳紀錄的各類資料
        return {m.name: m.result() for m in self.metrics}


In [ ]:
import numpy as np

# 透過前面的客製化訓練架構，放入對應的內容
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=3)

Epoch 1/3
32/32 [==============================] - 1s 1ms/step - loss: 2.0655 - mae: 1.3531
Epoch 2/3
32/32 [==============================] - 0s 902us/step - loss: 0.9961 - mae: 0.8835
Epoch 3/3
32/32 [==============================] - 0s 973us/step - loss: 0.4107 - mae: 0.5239


In [ ]:
#更客製化的訓練

loss_tracker = keras.metrics.Mean(name="loss")
mae_metric = keras.metrics.MeanAbsoluteError(name="mae")

class CustomModel(keras.Model):
    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True) 
            # 計算損失
            loss = keras.losses.mean_squared_error(y, y_pred)

        # 計算梯度
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # 更新權重
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # 更新與紀錄各類衡量標準，包含準確率、損失
        loss_tracker.update_state(loss)
        mae_metric.update_state(y, y_pred)
        return {"loss": loss_tracker.result(), "mae": mae_metric.result()}

    @property
    def metrics(self):
      #可以放各類衡量標準
        return [loss_tracker, mae_metric]


# 透過前面的客製化訓練架構，放入對應的內容
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)

# 我們不用在這裡寫上損失、指標（因為Class裡面已經包含）
model.compile(optimizer="adam")

x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=5)

Epoch 1/5
32/32 [==============================] - 0s 943us/step - loss: 0.3660 - mae: 0.4841
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2642 - mae: 0.4111
Epoch 3/5
32/32 [==============================] - 0s 849us/step - loss: 0.2558 - mae: 0.4049
Epoch 4/5
32/32 [==============================] - 0s 952us/step - loss: 0.2473 - mae: 0.3993
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2387 - mae: 0.3912
